# Classification of the Feature Neurons from the SNN
This notebook performs the classification of the feature neurons from the SNN according to certain criteria. In the future, it would be interesting to implement a 3rd layer in the SNN to classify the feature neurons automatically. For now, this manual classification will classify each neuron as one of the following:
- **Silent Neuron**: Neuron that does not fire at all.
- **Noisy Neuron**: Neuron that fires randomly or without a relevant pattern.
- **Ripple Neuron**: Neuron that fires in the presence of a ripple.
- **Fast Ripple Neuron**: Neuron that fires in the presence of a fast ripple.

## Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn


## Load the Voltage and Current Dynamics during the SNN run

#### Load the Baseline Thresholds from the output file from the baseline process

In [2]:
from utils.input import MarkerType, band_to_file_name, BaselineAlgorithm

# Specify the chosen Baseline Algorithm
chosen_baseline_alg_suffix = BaselineAlgorithm.MEAN

In [3]:
import numpy as np
from utils.io import preview_np_array

DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"

# Load the Baseline Thresholds
BASELINE_FILE = f"../signal_to_spike/baseline_results/{DATASET_FILENAME}_thresholds_{chosen_baseline_alg_suffix}.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)

Ripple Thresholds:  4.7916 -4.7916
FR Thresholds:  2.3014 -2.3014


In [4]:
from utils.input import MarkerType

def band_to_thresholds(band: MarkerType):
    """Returns the ripple and FR thresholds for the given band

    Args:
        band (MarkerType): the band for which the thresholds are required
    """
    if band == MarkerType.RIPPLE:
        return ripple_thresh_up, ripple_thresh_down
    elif band == MarkerType.FAST_RIPPLE:
        return fr_thresh_up, fr_thresh_down
    else:
        raise Exception("Invalid band type")

In [5]:
# Declare if using ripples, fast ripples, or both
chosen_band = MarkerType.FAST_RIPPLE     # RIPPLE, FAST_RIPPLE, or BOTH
band_file_name = band_to_file_name(chosen_band)

# Define the chosen thresholds
thresh_up, thresh_down = band_to_thresholds(chosen_band)
print("Thresholds: ", thresh_up, thresh_down)

INPUT_PATH = f"./results/{DATASET_FILENAME}"
TIME_SUFFIX = "time0-120000-1"
THRESH_SUFFIX = f"thresh{thresh_up}-{thresh_down}"

# Load the voltage and current data from the numpy files
voltage_file_name = f"{INPUT_PATH}/{band_file_name}_v_dynamics_0.07dv_5ch_{TIME_SUFFIX}_{THRESH_SUFFIX}.npy"
current_file_name = f"{INPUT_PATH}/{band_file_name}_u_dynamics_0.07dv_5ch_{TIME_SUFFIX}_{THRESH_SUFFIX}.npy"

v_dynamics = np.load(voltage_file_name)
u_dynamics = np.load(current_file_name)

preview_np_array(v_dynamics, "v_dynamics", edge_items=3)
preview_np_array(u_dynamics, "u_dynamics", edge_items=3)

Thresholds:  2.3014 -2.3014
v_dynamics Shape: (120000, 256).
Preview: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-3.72351870e-31 -2.78597523e-31 -4.30772885e-31 ... -3.53191573e-31
  -2.73483871e-31 -1.87360525e-31]
 [-3.46287239e-31 -2.59095697e-31 -4.00618783e-31 ... -3.28468163e-31
  -2.54340000e-31 -1.74245289e-31]
 [-3.22047132e-31 -2.40958998e-31 -3.72575469e-31 ... -3.05475391e-31
  -2.36536200e-31 -1.62048118e-31]]
u_dynamics Shape: (120000, 256).
Preview: [[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.000000

## Load the SNN Configuration and extract the fields
The SNN configuration contains:
- Numpy array with the ground_truth for each timestep
- Initial Time Offset
- Virtual Time Step Interval
- Number of Steps

In [6]:
from utils.snn import SNNSimConfig

# Load the SNN Config data
snn_config_file_name = f"{INPUT_PATH}/{band_file_name}_snn_config_{TIME_SUFFIX}_{THRESH_SUFFIX}.npy"

# Load the SNNConfig data as an element of the class SNNConfig
snn_config: SNNSimConfig = np.load(snn_config_file_name, allow_pickle=True).item()

# Extract the data fields from the SNNConfig object
ground_truth: np.ndarray = snn_config.ground_truth
init_offset = snn_config.init_offset
virtual_time_step_interval = snn_config.virtual_time_step_interval
num_steps = snn_config.num_steps

preview_np_array(ground_truth, "ground_truth", edge_items=3)
np.count_nonzero(ground_truth)

print("init_offset: ", init_offset)
print("virtual_time_step_interval: ", virtual_time_step_interval)
print("num_steps: ", num_steps)

ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]
init_offset:  0
virtual_time_step_interval:  1
num_steps:  120000


## Find the timesteps where the network spiked
Let's find the timesteps where the network spiked and create a `dictionary` mapping each feature neuron to the timesteps where it spiked.

In [7]:
from utils.data_analysis import find_spike_times

# Create a map storing the spike times for each feature neuron
neuron_spike_times = {}

# Call the find_spike_times util function that detects the spikes in a voltage array
spike_times_lif1 = find_spike_times(v_dynamics, u_dynamics)

total_spikes_count = 0
for (spike_time, neuron_idx) in spike_times_lif1:
    # Calculate the spike time in ms
    real_spike_time = init_offset + spike_time * virtual_time_step_interval

    # If the neuron index is not in the map (first time the feature neuron spikes), add it
    if neuron_idx not in neuron_spike_times:
        neuron_spike_times[neuron_idx] = [real_spike_time]
    # Otherwise, append the spike time to the list of spike times for that neuron
    else:
        neuron_spike_times[neuron_idx].append(real_spike_time)
    total_spikes_count += 1

    # print(f"Spike time: {real_spike_time} (iter. {spike_time}) at neuron: {neuron_idx}")

# Print the spike times for each feature neuron
print("neuron_spike_times: ", neuron_spike_times)


neuron_spike_times:  {9: [1009, 4130, 4774, 6685, 6699, 9689, 10513, 11627, 14061, 16798, 19756, 20152, 21775, 22184, 22655, 24660, 25120, 26647, 27053, 27067, 28520, 30255, 30266, 30760, 34606, 37188, 41232, 42297, 44095, 45153, 45538, 46640, 47659, 51041, 51562, 53306, 53653, 54239, 55020, 57299, 57555, 60712, 61153, 62237, 62633, 63299, 64636, 65661, 66289, 66614, 67648, 68030, 69142, 69776, 69798, 70763, 71601, 71611, 73162, 74064, 76250, 76628, 78755, 79272, 81745, 83658, 84194, 84716, 85767, 85776, 86270, 86634, 86645, 88855, 89042, 91567, 94021, 94620, 95631, 95649, 96261, 96700, 96709, 98056, 98065, 98653, 99280, 100569, 102697, 105792, 105799, 106035, 108533, 109516, 110309, 113618, 114291, 116778], 2: [1010, 3778, 3784, 4131, 4140, 4145, 4152, 7218, 7220, 10514, 11114, 11118, 11125, 11130, 12122, 12137, 12682, 12685, 12696, 13199, 13206, 14053, 14057, 14069, 14580, 14594, 15213, 15220, 15561, 15566, 17039, 18124, 18135, 20163, 20752, 20755, 20762, 20766, 21178, 22185, 22663, 

## Define the Time Window after the event insertion to consider as part of the event

In [8]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Define the time window after an HFO insertion to consider as part of the event
confidence_window = RIPPLE_CONFIDENCE_WINDOW
if chosen_band == MarkerType.FAST_RIPPLE:
    confidence_window = FR_CONFIDENCE_WINDOW
elif chosen_band == MarkerType.BOTH:
    confidence_window = BOTH_CONFIDENCE_WINDOW

## Merge the spikes that occur at a distance less than the confidence window
In order to calculate the efficiency of the feature neurons, we need to merge the spikes that occur at a distance less than the time window. Otherwise, we would be counting the same event multiple times, which would lead to an overestimation of the efficiency.

This is **probably** safe, since each annotated event occurs at a distance greater than the considered time window, usually at least 1 order of magnitude greater.

In [9]:
# Merge the spikes that occur at a distance less than the confidence window
merged_spikes_counter = 0
for neuron_key in neuron_spike_times.keys():
    spike_times = neuron_spike_times[neuron_key]

    # Array with the merged spike times
    merged_spike_times = [spike_times[0]]

    curr_idx = 1
    comparison_idx = 0
    while curr_idx < len(spike_times):
        spike_time = spike_times[curr_idx]
        prev_spike_time = spike_times[comparison_idx]

        if spike_time - prev_spike_time < confidence_window:
            # This spike is part of the same event
            curr_idx += 1
            merged_spikes_counter += 1
            continue
        else:
            # This spike is part of a new event
            merged_spike_times.append(spike_time)
            comparison_idx = curr_idx   # Update the comparison index to the new spike time
            curr_idx += 1

    # Update the neuron_spike_times map with the merged spike times
    neuron_spike_times[neuron_key] = merged_spike_times

# Print the spike times for each feature neuron
print("Merged neuron_spike_times: ", neuron_spike_times)

print(f"Merged a total of {merged_spikes_counter} spikes out of {total_spikes_count} spikes")

Merged neuron_spike_times:  {9: [1009, 4130, 4774, 6685, 9689, 10513, 11627, 14061, 16798, 19756, 20152, 21775, 22184, 22655, 24660, 25120, 26647, 27053, 28520, 30255, 30760, 34606, 37188, 41232, 42297, 44095, 45153, 45538, 46640, 47659, 51041, 51562, 53306, 53653, 54239, 55020, 57299, 57555, 60712, 61153, 62237, 62633, 63299, 64636, 65661, 66289, 66614, 67648, 68030, 69142, 69776, 70763, 71601, 73162, 74064, 76250, 76628, 78755, 79272, 81745, 83658, 84194, 84716, 85767, 86270, 86634, 88855, 89042, 91567, 94021, 94620, 95631, 96261, 96700, 98056, 98653, 99280, 100569, 102697, 105792, 106035, 108533, 109516, 110309, 113618, 114291, 116778], 2: [1010, 3778, 4131, 7218, 10514, 11114, 12122, 12682, 13199, 14053, 14580, 15213, 15561, 17039, 18124, 20163, 20752, 21178, 22185, 22663, 23025, 23640, 24196, 25135, 25770, 26283, 26660, 27054, 28521, 29263, 30256, 30741, 31014, 31727, 33082, 33722, 34043, 35104, 37196, 37541, 38200, 38547, 41240, 41624, 42284, 42708, 43306, 44096, 44533, 46269, 46

## Iterate over the SNN running time and classify the feature neurons

Create a numpy array with the classification of each feature neuron.

In [10]:
from utils.snn import NeuronClass

# Create numpy array containing the class of each feature neuron
feature_neuron_class = np.full(shape=(v_dynamics.shape[1]), fill_value=NeuronClass.SILENT)
preview_np_array(feature_neuron_class, "feature_neuron_class", edge_items=3)

feature_neuron_class Shape: (256,).
Preview: [0 0 0 ... 0 0 0]


### Keep track of the relevant events each feature neuron detected

In [11]:
# Define a map to keep track of the number of relevant events each feature neuron detected
relevant_neuron_spike_times = {}

# Create an empty list for each feature neuron with spikes
for neuron_idx in neuron_spike_times.keys():
    relevant_neuron_spike_times[neuron_idx] = []

print("relevant_neuron_spike_times: ", relevant_neuron_spike_times)

relevant_neuron_spike_times:  {9: [], 2: [], 112: [], 81: [], 0: [], 1: [], 3: [], 62: [], 8: [], 41: [], 6: [], 5: [], 10: [], 76: [], 20: [], 13: [], 54: [], 49: [], 12: [], 25: [], 186: [], 109: [], 88: [], 60: [], 4: [], 65: [], 7: [], 47: [], 105: [], 45: [], 181: [], 241: [], 219: [], 24: [], 231: [], 200: [], 73: [], 210: [], 251: [], 174: [], 42: [], 31: [], 27: [], 86: [], 35: [], 153: [], 43: [], 30: [], 188: [], 133: [], 53: [], 71: [], 15: [], 70: [], 132: [], 104: [], 48: [], 242: [], 239: [], 111: [], 32: [], 145: [], 169: [], 93: [], 218: [], 208: [], 120: [], 34: [], 19: [], 51: [], 154: [], 226: [], 119: [], 124: [], 138: [], 38: [], 177: [], 29: [], 94: [], 39: [], 191: [], 140: [], 66: [], 61: [], 125: [], 196: []}


### Define method that searches for a relevant event in the ground_truth np array

In [12]:
# Declare a sorted np.array of the ground truth events by timestamp
ground_truth_timestamps = np.array([ann_event[1] for ann_event in ground_truth])

preview_np_array(ground_truth_timestamps, "ground_truth_timestamps", edge_items=3)

ground_truth_timestamps Shape: (199,).
Preview: [  1000.     3206.54   3770.02 ... 116096.   116769.   119000.  ]


In [13]:
def has_relevant_event(spike_time, confidence_window) -> float | None:
    """
    Searches for a relevant event in the ground truth array given a timestamp.
    The annotated event must be located before the spike time, since the annotation
    corresponds to the insertion of the event.
    Thus, the Event must be in the window [spike_time - confidence_window, spike_time]

    Returns:
    - The insertion time of the relevant event if found
    - None if no relevant event was found
    """
    # Check if spike_time is greater than the first event in the ground truth. If not, return False
    if ground_truth_timestamps[0] > spike_time:
        return None

    for event_time in ground_truth_timestamps:
        if spike_time - confidence_window <= event_time <= spike_time:
            return event_time
        
        if event_time > spike_time:
            # Since the events are sorted, if the event_time is greater than the spike_time, we can stop looking
            return None
    

In [14]:
# Iterate over the feature neurons that spiked
for neuron_idx in neuron_spike_times.keys():
    curr_spike_times = neuron_spike_times[neuron_idx]
    
    # For each Spike
    for curr_spike_time in curr_spike_times:
        # Since the SNN will spike after the insertion of the event, we need to consider the confidence window before the SNN spike.
        # The Annotated event is located at the insertion time (before)

        # Check if the ground truth contains at least 1 annotated event within the confidence window
        if has_relevant_event(curr_spike_time, confidence_window) != None:
            # Add the event to the relevant neuron spike times
            relevant_neuron_spike_times[neuron_idx].append(curr_spike_time)

print("relevant_neuron_spike_times: ", relevant_neuron_spike_times)

relevant_neuron_spike_times:  {9: [1009, 9689, 10513, 14061, 16798, 19756, 20152, 21775, 24660, 26647, 27053, 30255, 30760, 41232, 42297, 45153, 45538, 46640, 51041, 51562, 53306, 53653, 55020, 57299, 60712, 61153, 63299, 64636, 66614, 67648, 69142, 70763, 71601, 73162, 74064, 76250, 76628, 78755, 81745, 83658, 84194, 84716, 85767, 86270, 89042, 94620, 96261, 98653, 99280, 102697, 105792, 108533, 109516, 110309, 113618, 116778], 2: [1010, 3778, 10514, 12682, 13199, 14053, 15213, 15561, 17039, 18124, 20163, 20752, 21178, 22663, 23025, 25135, 26283, 26660, 27054, 30256, 31014, 33082, 33722, 35104, 37196, 37541, 38200, 38547, 41240, 41624, 42284, 42708, 43306, 44533, 46632, 47669, 48229, 50302, 51038, 52125, 53293, 56097, 57300, 57571, 58572, 59704, 61168, 63800, 65677, 66308, 67235, 67645, 68044, 69129, 71302, 72717, 73172, 73735, 74056, 75164, 76230, 76642, 79284, 79540, 80240, 81737, 82062, 83661, 84197, 85126, 85768, 86259, 88167, 90136, 90649, 91254, 94033, 94629, 96276, 96701, 97221

## Statistics regarding the spiking neurons

In [15]:
# Create an array containing the relevant spike ratio for each feature neuron 
# in the order of their keys
relevant_spike_ratios = []
for neuron_idx in neuron_spike_times.keys():
    # Get all the spikes of the current neuron
    curr_spike_times = neuron_spike_times[neuron_idx]
    # Get the relevant spikes of the current neuron
    relevant_spike_times = relevant_neuron_spike_times[neuron_idx]

    # Calculate the ratio of relevant spikes
    relevant_spike_ratio = (len(relevant_spike_times) / len(curr_spike_times)) * 100 if len(curr_spike_times) > 0 else 0.0
    relevant_spike_ratios.append(relevant_spike_ratio)

In [16]:
PLOT_RELEVANCY_STATS = True

if PLOT_RELEVANCY_STATS:
    # Iterate over the feature neurons that spiked
    for iter_idx, neuron_idx in enumerate(neuron_spike_times.keys()):
        # Get all the spikes of the current neuron
        curr_spike_times = neuron_spike_times[neuron_idx]

        # Get the relevant spikes of the current neuron
        relevant_spike_times = relevant_neuron_spike_times[neuron_idx]

        print(f"Neuron {neuron_idx}: ")
        print(f"Spikes: {curr_spike_times}")
        print(f"Relevant Spikes: {relevant_spike_times}")
        print(f"Relevant Spikes Ratio: {relevant_spike_ratios[iter_idx]}%")
        print("================================================================\n")

Neuron 9: 
Spikes: [1009, 4130, 4774, 6685, 9689, 10513, 11627, 14061, 16798, 19756, 20152, 21775, 22184, 22655, 24660, 25120, 26647, 27053, 28520, 30255, 30760, 34606, 37188, 41232, 42297, 44095, 45153, 45538, 46640, 47659, 51041, 51562, 53306, 53653, 54239, 55020, 57299, 57555, 60712, 61153, 62237, 62633, 63299, 64636, 65661, 66289, 66614, 67648, 68030, 69142, 69776, 70763, 71601, 73162, 74064, 76250, 76628, 78755, 79272, 81745, 83658, 84194, 84716, 85767, 86270, 86634, 88855, 89042, 91567, 94021, 94620, 95631, 96261, 96700, 98056, 98653, 99280, 100569, 102697, 105792, 106035, 108533, 109516, 110309, 113618, 114291, 116778]
Relevant Spikes: [1009, 9689, 10513, 14061, 16798, 19756, 20152, 21775, 24660, 26647, 27053, 30255, 30760, 41232, 42297, 45153, 45538, 46640, 51041, 51562, 53306, 53653, 55020, 57299, 60712, 61153, 63299, 64636, 66614, 67648, 69142, 70763, 71601, 73162, 74064, 76250, 76628, 78755, 81745, 83658, 84194, 84716, 85767, 86270, 89042, 94620, 96261, 98653, 99280, 102697,

## Plot the Relevant Spike Ratio of the Feature Neurons that Spike

In [17]:
# Create a bar plot containing the % of relevant spikes for each feature neuron 
from utils.bar_plot import create_bar_fig  # Import the function to create the figure

# Define the x and y values
neuron_label = [f"Neu. {neuron_idx}" for neuron_idx in neuron_spike_times.keys()]

# sorting the bars means sorting the range factors
neurons_descending_rel_ratio = sorted(neuron_label, key=lambda x: relevant_spike_ratios[neuron_label.index(x)], reverse=True)

# Create the LIF1 Voltage
feat_neurons_relevancy_plot = create_bar_fig(
    title="Feature Neurons Relevant Spikes Ratio", 
    x_axis_label='Feature Neuron Index', 
    y_axis_label='Relevant Spike Ratio (%)',
    x=neuron_label,
    y=relevant_spike_ratios,
    x_range=neurons_descending_rel_ratio,
    sizing_mode="stretch_width",
    bar_width=0.5,
    tooltips=[("Neuron Index", "@x"), ("Relevant Spike Ratio", "@top%")]
)

In [18]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    # Show the plot
    bplt.show(feat_neurons_relevancy_plot)

In [19]:
EXPORT_RELEVANCY_PLOT = True
OUTPUT_FOLDER = f"./neuron_classification/{DATASET_FILENAME}"

if EXPORT_RELEVANCY_PLOT:
    # Create the output folder if it does not exist
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    file_path = f"{OUTPUT_FOLDER}/{band_file_name}_relevancy_barplot_0.07dv_{TIME_SUFFIX}_{THRESH_SUFFIX}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="Feature Neurons Relevancy (%) Bar Plot")

    # Save the plot
    bplt.save(feat_neurons_relevancy_plot)

## Classify the Feature Neurons according to their Spiking Activity
In this step, we will update the `feature_neuron_class` array with the classification of the `Noisy Neurons`, `Ripple Neurons` and `Fast Ripple Neurons`.

We set a **threshold of 0.9** for the `Relevant Spike Ratio` to classify a neuron as a `Ripple` or `Fast Ripple` Neuron.

In [20]:
from utils.snn import marker_type_to_neuron_class

relevant_ratio_threshold = 90.0

# Iterate over the feature neurons that spiked
for iter_idx, neuron_idx in enumerate(neuron_spike_times.keys()):
    # Get all the spikes of the current neuron
    curr_spike_times = neuron_spike_times[neuron_idx]
    # Get the relevant spikes of the current neuron
    relevant_spike_times = relevant_neuron_spike_times[neuron_idx]
    # Get the relevant spikes ratio of the current neuron
    relevant_spike_ratio = relevant_spike_ratios[iter_idx]

    if relevant_spike_ratio >= relevant_ratio_threshold:
        # Classify the Neuron as a RIPPLE/FAST RIPPLE/BOTH detector based on the band we are analyzing
        feature_neuron_class[neuron_idx] = marker_type_to_neuron_class(chosen_band)
    else:
        # Classify the Neuron as Noisy
        feature_neuron_class[neuron_idx] = NeuronClass.NOISY

### Show the classification of the Feature Neurons

In [21]:
preview_np_array(feature_neuron_class, "feature_neuron_class", edge_items=3)

feature_neuron_class Shape: (256,).
Preview: [1 1 1 ... 0 0 0]


# Calculate the Classification Metrics of the SNN based on the Feature Neurons
As of right now, we have a Spiking Neural Network that updates it's state in real-time. This network is capable of detecting relevant oscillatory events through spikes.

Therefore, we have to define the conditions that identify the detection of a HFO event or not. Since each Feature Neuron is modelled to be tuned to a specific frequency range, not all of them will fire simultaneously in the presence of a HFO event. Hence, we need to define a threshold for the number of Feature Neurons that need to fire in order to classify an event as a HFO. This is an exploratory approach, but as a starting point, let's say that we need 2 neurons firing during the time window (`confidence_window`).

In [22]:
# Initialize the variables that store the values of the Confusion Matrix
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

num_consensus_neurons = 1 # 2   # Number of neurons that must agree to classify an event as relevant

### Build a numpy array containing all the spike times from the `Detector Neurons` ordered by spiking time

In [23]:
# Find the index of the Feature Neurons that are HFO(Ripple/FR/Both) Detectors
detector_indices = np.where(feature_neuron_class == marker_type_to_neuron_class(chosen_band))[0]
preview_np_array(detector_indices, "detector_indices")

detector_indices Shape: (51,).
Preview: [ 12  15  19  20  24 ... 226 231 239 242 251]


In [24]:
# Build an ordered list of spike times from all the HFO Detector Neurons
detector_spike_times = []

for neuron_idx in detector_indices:
    # print(f"Adding spikes from neuron {neuron_idx}: {neuron_spike_times[neuron_idx]}")
    detector_spike_times.extend(neuron_spike_times[neuron_idx])

# Sort the list of spike times
detector_spike_times.sort()

detector_spike_times = np.array(detector_spike_times)
preview_np_array(detector_spike_times, "detector_spike_times")

detector_spike_times Shape: (306,).
Preview: [  3214   3785   4135   4786   4793 ... 116768 116780 116783 119011 119016]


In [25]:
import math

curr_detector_idx = 0   # Index of the current detector neuron spike
curr_ground_truth_idx = 0   # Index of the current ground truth event in the ordered array

# Stores an active annotation. If None, there is no active annotation
# An annotation is active in the interval [annotation_start, annotation_start + confidence_window]
active_annotation = None    # Timestamp of the active annotation   

# Go through the voltage dynamics array and calculate the classification metrics of the network
for run_step, feat_neurons_v in enumerate(v_dynamics):
    is_end_detector_spikes = curr_detector_idx >= len(detector_spike_times)
    is_end_ground_truth = curr_ground_truth_idx >= len(ground_truth)

    # Block that checks if a new annotation gets activated
    if not is_end_ground_truth:
        # Check if the next annotation was inserted at this run_step
        next_annotation_insertion = math.ceil(ground_truth_timestamps[curr_ground_truth_idx])
        if next_annotation_insertion == run_step:
            if active_annotation != None:
                raise Exception("There is already an active annotation. This should not happen.")

            # Start a new active annotation
            active_annotation = next_annotation_insertion

            # Increment the curr_ground_truth_idx since this annotation will be processed
            curr_ground_truth_idx += 1

    # If there are still spikes from the Detector Neurons
    if not is_end_detector_spikes:
        next_spike_iter = detector_spike_times[curr_detector_idx]
        # If the next spike occurs in the current run_step interval, let's check what kind of prediction it is
        if next_spike_iter == run_step:
            # Increment the curr_detector_idx since this spike will be processed
            curr_detector_idx += 1

            # Check if there is an annotated event in the ground_truth within the confidence window [run_step, run_step-confidence_window]
            if active_annotation != None:
                # If there is a relevant event, let's see if the SNN predicted it.
                num_votes = 1   # The first vote is the current spike (SNN needs num_consensus_neurons to classify the event as relevant)

                # Calculate the window of detection for this event
                # The annotated events are floats, so we need to ceil the end_relevant_event_time because 
                # we are comparing with the integer run_step (iteration index of the simulation) (TODO: Include the virtual_step in the calculation)
                end_relevant_event_time = math.ceil(active_annotation + confidence_window)

                # Check if there are more spikes in the confidence window
                while curr_detector_idx < len(detector_spike_times) and detector_spike_times[curr_detector_idx] <= end_relevant_event_time:
                    num_votes += 1
                    curr_detector_idx += 1

                # If the number of votes is greater than the consensus threshold, classify the event as relevant
                if num_votes >= num_consensus_neurons:
                    # The SNN detected the event -> True Positive
                    true_positive += 1
                else:
                    # The SNN didn't detect the event (Not enough votes) -> False Negative
                    # False Negatives may also occur without any spikes in the network
                    false_negative += 1

                # The current annotation was processed, so we can deactivate it
                active_annotation = None
            else:
                # If there is no annotated event in the confidence window, let's check if the SNN predicted one.
                # If the SNN predicted an event, it is a False Positive
                # Calculate the window that would cause an SNN Detection
                end_relevant_event_iter = next_spike_iter + confidence_window

                # Verify if there isn't an annotated event between the current run_step and the end_relevant_event_iter
                # If there is an annotated event in between, we need to modify the end_relevant_event_iter to the annotated event time-1
                middle_relevant_event_time = has_relevant_event(end_relevant_event_iter, confidence_window)
                if middle_relevant_event_time != None:
                    # If there is an annotated event in the confidence window, we need to update the end of the window
                    # to the timestep before the annotated event, since we are analyzing the prediction of a False Positive
                    end_relevant_event_iter = math.ceil(middle_relevant_event_time) - 1 

                # Check if there are more spikes in the confidence window
                while curr_detector_idx < len(detector_spike_times) and detector_spike_times[curr_detector_idx] <= end_relevant_event_iter:
                    num_votes += 1
                    curr_detector_idx += 1

                # If the number of votes is greater than the consensus threshold
                if num_votes >= num_consensus_neurons:
                    # The SNN detected an event that does not exist -> False Positive
                    false_positive += 1
                else:
                    # The SNN didn't detect any event -> True Negative
                    true_negative += 1
            
            # Go to the next iteration
            continue
    
    # If one of the 2 conditions is true:
    #   1. There are no more spikes from the Detector Neurons
    #   2. The current run_step does not contain a spike from the Detector Neurons
    # Then: let's check if the active annotation is still active and report a True Negative or False Negative
    # according to the answer.
    
    # Block that checks if the active annotation is still active
    if active_annotation != None:
        if run_step >= active_annotation + confidence_window + 1:   # The +1 is to consider an extra iteration because the annotation is a float ceiled
            # The active annotation is over
            active_annotation = None

            # The SNN didn't detect the event in the interval [active_annotation, active_annotation + confidence_window]
            # Thus, it is a False Negative
            false_negative += 1
        else:
            # The active annotation is still active, so it is a True Negative (No event detected)
            true_negative += 1
    else:
        # If there is no active annotation, the SNN didn't detect any event rightfully so
        # So, it is a True Negative
        true_negative += 1


    # Edge case: If there is an annotated event at a distance < confidence_window from the end of the simulation, the feature events
    # won't have time to detect it. Thus, we don't consider that event.
    # TODO: Consider this edge case?

## Show the Confusion Matrix

In [26]:
# Print the Confusion Matrix
print("True Positive: ", true_positive)
print("False Positive: ", false_positive)
print("True Negative: ", true_negative)
print("False Negative: ", false_negative)

# Print the Total of predictions
total_predictions = true_positive + false_positive + true_negative + false_negative
print("Total Predictions: ", total_predictions)

True Positive:  168
False Positive:  10
True Negative:  119791
False Negative:  31
Total Predictions:  120000


## Calculate Classification Metrics

In [27]:
# Calculate relevant metrics
accuracy = (true_positive + true_negative) / total_predictions * 100    # Proportion of correct predictions
precision = true_positive / (true_positive + false_positive) * 100      # Proportion of TPs that were identified correctly
recall = true_positive / (true_positive + false_negative) * 100         # Proportion of TPs that were captured by the model
f1_score = (2 * precision * recall) / (precision + recall)              # Harmonic mean of Precision and Recall
specificity = true_negative / (true_negative + false_positive) * 100    # Proportion of TNs that were identified correctly

print(f"Accuracy: {accuracy}%")
print(f"Precision: {precision}%")
print(f"Recall: {recall}%")
print(f"F1 Score: {f1_score}")
print(f"Specificity: {specificity}%")

Accuracy: 99.96583333333334%
Precision: 94.3820224719101%
Recall: 84.42211055276381%
F1 Score: 89.12466843501326
Specificity: 99.99165282426691%


# Export the results of the Classification to a JSON file
Export the results of the classification to a JSON file. This file will include:
- Frequency Band used (`Ripple`, `Fast Ripple` or `Both`)
- `Confidence Window` used
- Classification of each Feature Neuron (`Silent`, `Noisy`, `Ripple`, `Fast Ripple`, `Both`)
- Number of `Consensus Feature Neurons` to classify an event as a HFO
- Classification Metrics (`True Positives`, `False Positives`, `True Negatives`, `False Negatives`, `Accuracy`, `Precision`, `Recall`, `F1 Score`, `Specificity`)

In [28]:
# Calculate the number of feature neurons of each class
num_silent_neurons = np.count_nonzero(feature_neuron_class == NeuronClass.SILENT)
num_noisy_neurons = np.count_nonzero(feature_neuron_class == NeuronClass.NOISY)
num_detector_neurons = np.count_nonzero(feature_neuron_class == marker_type_to_neuron_class(chosen_band))

print(f"Number of Silent Neurons: {num_silent_neurons}")
print(f"Number of Noisy Neurons: {num_noisy_neurons}")
print(f"Number of Detector Neurons: {num_detector_neurons}")
print(f"Total: {num_silent_neurons + num_noisy_neurons + num_detector_neurons}")

Number of Silent Neurons: 170
Number of Noisy Neurons: 35
Number of Detector Neurons: 51
Total: 256


In [29]:
import json

# Export the results to a JSON file

# Create a dictionary with the results
json_results = {
    "freq_band": band_file_name,
    "confidence_window": confidence_window,
    "feature_neuron_class": {
        "array": feature_neuron_class.tolist(),
        "counts": {
            "silent_neurons": num_silent_neurons,
            "noisy_neurons": num_noisy_neurons,
            "detector_neurons": num_detector_neurons
        },
    },
    "num_consensus_neurons": num_consensus_neurons,
    "metrics": {
        "true_positive": true_positive,
        "false_positive": false_positive,
        "true_negative": true_negative,
        "false_negative": false_negative,
        "total_predictions": total_predictions,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "specificity": specificity
    }
}

EXPORT_JSON_FILE = True
if EXPORT_JSON_FILE:
    json_file_name = f"{OUTPUT_FOLDER}/{band_file_name}_results_0.07dv_{TIME_SUFFIX}_{THRESH_SUFFIX}_classif{num_consensus_neurons}.json"
    with open(json_file_name, 'w') as f:
        json.dump(json_results, f)